In [1]:
import pandas as pd

1.校验csv文件名称中提取的store_code是否满足:”JD_数字“的格式

In [15]:
import re
import os

for filename in os.listdir("./item"):
    if filename.endswith(".csv"):
        if re.match("^JD_(\d+)_(.*?).csv$", filename):
            print(f"{filename}符合条件")
        else:
            print(f"{filename}不符合条件")


JD_00001_导出SPU_2023_09_12_0.csv符合条件
JD_00001_导出SPU_2023_09_13.csv符合条件
XD_00001_导出.csv不符合条件


In [4]:
# 读取文件
data_1 = pd.read_csv('./item/JD_00001_导出SPU_2023_09_12_0.csv', encoding='gbk')

data_1.sample()

,商品编码,商品名称,京东分类,品牌,运费模板,商品货号,成本价,市场价,京东价,库存,...,默认好评数,好评率,中评数（3星，2星）,中评率,差评数（1星）,差评率,晒单数,追评数,无理由退货,是否指定快递配送
2,198950342,耐克（NIKE）男士防滑运动训练跑步鞋透气吸汗缓震轻便休闲慢跑鞋,运动户外 > 运动鞋包 > 跑步鞋,耐克（NIKE）,未选择运费模板,02FE52D0,1575.1,2487.0,1658,102,...,0,100.00%,"0(0,0)",0.00%,0,0.00%,0,0,不支持无理由退货,否


In [7]:
# 这个文件坏了
data_2 = pd.read_csv('./item/JD_00001_导出SPU_2023_09_13.csv', encoding='utf-8')
data_2.sample()

,��������,Ʒ��,�˷�ģ��,��Ʒ����,�ɱ���,�г���,������,���,ë�أ�kg��,���ڷ���,...,Ĭ�Ϻ�����,������.2,��������3�ǣ�2�ǣ�,������.3,��������1�ǣ�,������.4,ɹ����,׷����,�������˻�,�Ƿ�ָ���������
0,�˶����� > �˶�Ь�� > �ܲ�Ь,�Ϳˣ�NIKE��,δѡ���˷�ģ��,02FE5BBA,1382.25,2182.5,1455,4,2,�Ϳ�-��ʿ,...,0,100.00%,"0(0,0)",0.00%,0,0.00%,0,0,��֧���������˻�,��


In [11]:
data_3 = pd.read_csv('./item/XD_00001_导出.csv', encoding='gbk')
data_3.sample()

,商品编码,商品名称,京东分类,品牌,运费模板,商品货号,成本价,市场价,京东价,库存,...,默认好评数,好评率,中评数（3星，2星）,中评率,差评数（1星）,差评率,晒单数,追评数,无理由退货,是否指定快递配送
3,198950847,耐克（NIKE）减震运动鞋跑步鞋男子徒步防滑学生鞋轻便透气旅游鞋,运动户外 > 运动鞋包 > 跑步鞋,耐克（NIKE）,未选择运费模板,02FE5B6C,4011.85,6334.5,4223,16,...,0,100.00%,"0(0,0)",0.00%,0,0.00%,0,0,不支持无理由退货,否


2. 校验csv文件是否包含配置文件中指定的所有列(csv文件列)，如果没有报错，并打印缺少的列名

In [16]:
# 读取.properties配置文件
with open("./item/column_mapping.propties", "r", encoding="utf-8") as f:
    property_list = {data.split("=")[0]: data.split("=")[1] for data in f.read().split("\n")}


In [26]:
# 检查第一个文件
for filename in os.listdir("./item"):
    if not filename.endswith(".csv") or filename == "JD_00001_导出SPU_2023_09_13.csv":
        continue
    tmp_data = pd.read_csv(f'./item/{filename}', encoding='gbk')
    result = list(filter(lambda x: x not in tmp_data.columns, property_list.keys()))
    if len(result) > 0:
        print(f"{filename}存在{len(result)}列不在数据内，数据是：{','.join([property_list['r'] for r in result])}")
    else:
        print(f"{filename}所有列都在文件内")


JD_00001_导出SPU_2023_09_12_0.csv所有列都在文件内
XD_00001_导出.csv所有列都在文件内


3. 根据配置文件中配置的映射将csv中的每一行转换为insert语句(注意将store_code一列也加入)，并存入sqlite3数据库，数据库表名为items，根据实际设计表


In [32]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///database.db', echo=False)

with engine.connect() as conn:
    data_1.to_sql(name='items', con=conn, if_exists="replace")
    data_3.to_sql(name='items', con=conn, if_exists="append")

ValueError: Table 'items' already exists.

4. 处理成功的csv文件，将文件重命名为： 原文件名称.COMPLETED；处理失败的csv文件，将文件重命名为： 原文件名称.FAILED

In [ ]:
# 文件重命名
os.rename("JD_00001_导出SPU_2023_09_12_0.csv", "JD_00001_导出SPU_2023_09_12_0.csv.COMPLETED")

os.rename("JD_00001_导出SPU_2023_09_13.csv", "JD_00001_导出SPU_2023_09_13.csv.FAILED")

os.rename("JD_00001_导出SPU_2023_09_12_0.csv", "JD_00001_导出SPU_2023_09_12_0.csv.COMPLETED")

In [12]:
from pandas.io.sql import get_schema
import pandas as pd

import re
import os

with open("./item/column_mapping.propties", "r", encoding="utf-8") as f:
    property_list = {data.split("=")[0]: data.split("=")[1] for data in f.read().split("\n")}

engine = create_engine('sqlite:///database.db', echo=False)


def generate_insert_statement(source: pd.DataFrame, target: str) -> list[str]:
    """
    生成insert语句
    :param source:数据源 
    :param target: 需要插入的表
    :return: 插入列表
    """
    sql_list = []
    for data in source.iterrows():
        sql_list.append(
            f"INSERT INTO  {target} ({','.join(source.columns)}) VALUES ({','.join(data.values)})"
        )

def generate_create_table_statement(source: pd.DataFrame, target: str) -> str:
    """
    生成建表语句
    :param source: 数据源 
    :param target: 数据表名
    :return: 
    """
    return get_schema(source.reset_index(), target)

for filename in os.listdir("./item"):
    if not filename.endswith(".csv"):
        continue

    match_result = re.match("^([A-Z]+_(\d+))_(.*?).csv$", filename)

    if match_result[1].startswith("JD_"):
        print(f"{filename}符合条件")
    else:
        print(f"{filename}不符合条件")

    # 读取文件
    try:
        tmp_data = pd.read_csv(f"./item/{filename}", encoding='gbk')
    except Exception as e:
        os.rename(filename, filename + ".FAILED")
        print(f"读取文件：{filename}出错，错误原因：{e}")
        continue

    result = list(filter(lambda x: x not in tmp_data.columns, property_list.keys()))
    if len(result) > 0:
        print(f"{filename}存在{len(result)}列不在数据内，数据是：{','.join([property_list['r'] for r in result])}")
    else:
        print(f"{filename}所有列都在文件内")

    # 建表
    # 添加store_code列
    tmp_data['store_code'] = match_result[1]

    tmp_data.rename(property_list)

    # with engine.connect() as conn:
    #     conn.execute(generate_create_table_statement(tmp_data, "items"))
    #     conn.execute(generate_insert_statement(tmp_data, "items"))

    # 生成insert语句
    with engine.connect() as conn:
        tmp_data.to_sql(name='items', con=conn, if_exists="append")

    os.rename(filename, filename + ".COMPLETED")


JD_00001
JD_00001_导出SPU_2023_09_12_0.csv符合条件
JD_00001
JD_00001_导出SPU_2023_09_13.csv符合条件
XD_00001
XD_00001_导出.csv不符合条件
